In [1]:
from itertools import chain

import numpy as np
import pandas as pd
import stanza
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

import utils

#stanza.download(lang="en", processors="tokenize")

In [6]:
df = utils.load_tokenized_dataset("datasets/yahoo_1k_sample_tokenized.csv")
tokens = df.tokens.apply(utils.flat_nested_list).sum()
tokens = [token.lower() for token in tokens]
vocabulary = utils.build_vocabulary(tokens, min_appearences=6)
encoder = tfds.features.text.TokenTextEncoder(vocabulary)

In [7]:
a = pd.read_csv("datasets/yahoo_1k_sample.csv")
text_ds = tf.data.Dataset.from_tensor_slices(a.text)


def encode(text_tensor):
    return tf.constant(encoder.encode(text_tensor.numpy()))


def encode_map_fn(text):
    # py_func doesn't set the shape of the returned tensors.
    encoded_text = tf.py_function(encode, inp=[text], Tout=tf.int32)

    # `tf.data.Datasets` work best if all components have a shape set
    #  so set the shapes manually: 
    encoded_text.set_shape([None])
    return encoded_text


encoded_ds = text_ds.map(encode_map_fn)

In [8]:
TRAIN_SIZE = 900
TEST_SIZE = 100
BUFFER_SIZE = 1_000
BATCH_SIZE = 50

train_data = encoded_ds.skip(TEST_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=[None])

test_data = encoded_ds.take(TEST_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=[None])

In [213]:
model = keras.Sequential([
     layers.Embedding(encoder.vocab_size, 16)
])
model.compile(optimizer="adam", loss=)

2020-03-25 10:46:06 WARNING: Output embedding_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to embedding_1.


ValueError: The model cannot be compiled because it has no loss to optimize.

In [12]:
encoder.vocab_size

308